# 01_data_analysis
Dieses Notebook ist für die Auswertung unserer Daten

In [1]:

from pathlib import Path
import numpy as np
import pandas as pd

project_root = Path('..').resolve()
output_dir = project_root / 'work/output'
processed_dir = output_dir / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)

energy_zh = pd.read_csv(processed_dir / 'energy_zh.csv', parse_dates=['energyreporter_date'])
wealth_small = pd.read_csv(processed_dir / 'wealth_small.csv')

print('Loaded prepared inputs', energy_zh.shape, wealth_small.shape)


Loaded prepared inputs (9004, 5) (17176, 6)


In [2]:

# Energiekennzahlen 2024 aggregieren
energy_2024 = energy_zh[energy_zh['energyreporter_date'].dt.year == 2024].copy()
consum = (
    energy_2024
    .groupby('bfs_nr', as_index=False)
    .agg({
        'elec_consumption_mwh_per_year_per_capita': 'mean',
        'elec_consumption_households_mwh_per_year_per_capita': 'mean'
    })
    .rename(columns={
        'elec_consumption_mwh_per_year_per_capita': 'total_elec_per_capita_mwh_2024',
        'elec_consumption_households_mwh_per_year_per_capita': 'household_elec_per_capita_mwh_2024'
    })
)

consum.to_csv(processed_dir / 'energy_consumption_2024.csv', index=False)
consum.head()


,bfs_nr,total_elec_per_capita_mwh_2024,household_elec_per_capita_mwh_2024
0,1,4.495275,2.970700
1,2,4.198583,1.678750
2,3,2.789025,1.738283
3,4,3.874125,2.251017
4,5,5.465508,2.169908


In [3]:
# Vermögenskennzahlen 2022 extrahieren
wealth_2022 = wealth_small[wealth_small['INDIKATOR_JAHR'] == 2022].copy()
patterns = {
    'income_median_2022': r'^Steuerb\. Einkommen.*Median',
    'income_avg_2022': r'^Steuerb\. Einkommen.*Durchschn',
    'income_total_per_capita_2022': r'^Steuerb\. Einkommen.*(je Einwohner|pro Kopf)',
    'capital_median_2022': r'^Steuerb\. Vermögen.*Median',
    'capital_avg_2022': r'^Steuerb\. Vermögen.*Durchschn',
    'capital_total_per_capita_2022': r'^Steuerb\. Vermögen.*(je Einwohner|pro Kopf)'
}
wealth_2022['metric'] = np.nan
for col, pattern in patterns.items():
    match_mask = wealth_2022['INDIKATOR_NAME'].str.contains(pattern, regex=True, na=False)
    wealth_2022.loc[match_mask, 'metric'] = col

wide = (
    wealth_2022.dropna(subset=['metric'])
    .pivot_table(index='BFS_NR', columns='metric', values='INDIKATOR_VALUE', aggfunc='first')
    .reset_index()
    .rename(columns={'BFS_NR': 'bfs_nr'})
)
wide.to_csv(processed_dir / 'wealth_metrics_2022.csv', index=False)
wide.head()

C:\Users\thier\AppData\Local\Temp\ipykernel_19672\3472260061.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'income_median_2022' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  wealth_2022.loc[match_mask, 'metric'] = col
C:\Users\thier\AppData\Local\Temp\ipykernel_19672\3472260061.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  match_mask = wealth_2022['INDIKATOR_NAME'].str.contains(pattern, regex=True, na=False)
C:\Users\thier\AppData\Local\Temp\ipykernel_19672\3472260061.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  match_mask = wealth_2022['INDIKATOR_NAME'].str.contains(pattern, regex=True, na=False)


metric,bfs_nr,capital_avg_2022,capital_median_2022,income_avg_2022,income_median_2022
0,0,491000.0,99000.0,78800.0,60100.0
1,1,851000.0,200000.0,96700.0,68800.0
2,2,334000.0,48000.0,62900.0,52400.0
3,3,432000.0,118000.0,80400.0,67100.0
4,4,523000.0,125000.0,78800.0,57200.0


In [4]:

# Energie- und Vermögensdaten kombinieren
municipality_lookup = energy_zh[['bfs_nr', 'municipality']].drop_duplicates(subset='bfs_nr')
merged = consum.merge(wide, on='bfs_nr', how='inner')
merged = merged.merge(municipality_lookup, on='bfs_nr', how='left')

# Fallback für Gemeindebezeichnungen aus der Vermögensdatei
wealth_names = wealth_small[['BFS_NR', 'GEBIET_NAME']].drop_duplicates(subset='BFS_NR')
wealth_names = wealth_names.rename(columns={'BFS_NR': 'bfs_nr', 'GEBIET_NAME': 'municipality_wealth'})
merged = merged.merge(wealth_names, on='bfs_nr', how='left')
if 'municipality' not in merged.columns:
    merged['municipality'] = merged['municipality_wealth']
else:
    merged['municipality'] = merged['municipality'].fillna(merged['municipality_wealth'])
merged = merged.drop(columns=['municipality_wealth'])

# Numerische Spalten bereinigen und Verhältnisse bilden
value_cols = [
    'income_median_2022','income_avg_2022','income_total_per_capita_2022',
    'capital_median_2022','capital_avg_2022','capital_total_per_capita_2022'
]
for col in value_cols:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors='coerce').replace(0, np.nan)

if 'capital_median_2022' in merged.columns:
    merged['ratio_consum_to_capital_median'] = merged['household_elec_per_capita_mwh_2024'] / merged['capital_median_2022']
if 'capital_avg_2022' in merged.columns:
    merged['ratio_consum_to_capital_avg'] = merged['household_elec_per_capita_mwh_2024'] / merged['capital_avg_2022']
if 'capital_total_per_capita_2022' in merged.columns:
    merged['ratio_consum_to_capital_total'] = merged['household_elec_per_capita_mwh_2024'] / merged['capital_total_per_capita_2022']

merged.to_csv(processed_dir / 'merged_energy_wealth.csv', index=False)
print(f"Merged dataset saved to {processed_dir / 'merged_energy_wealth.csv'}")
merged.head()


Merged dataset saved to C:\dev\python\04-zhaw-scripting-wealth-energy-analysis-zurich\04-zhaw-scripting-wealth-energy-analysis-zurich\work\output\processed\merged_energy_wealth.csv


,bfs_nr,total_elec_per_capita_mwh_2024,household_elec_per_capita_mwh_2024,capital_avg_2022,capital_median_2022,income_avg_2022,income_median_2022,municipality,ratio_consum_to_capital_median,ratio_consum_to_capital_avg
0,1,4.495275,2.970700,851000.0,200000.0,96700.0,68800.0,Aeugst am Albis,0.000015,0.000003
1,2,4.198583,1.678750,334000.0,48000.0,62900.0,52400.0,Affoltern am Albis,0.000035,0.000005
2,3,2.789025,1.738283,432000.0,118000.0,80400.0,67100.0,Bonstetten,0.000015,0.000004
3,4,3.874125,2.251017,523000.0,125000.0,78800.0,57200.0,Hausen am Albis,0.000018,0.000004
4,5,5.465508,2.169908,556000.0,111000.0,83100.0,62800.0,Hedingen,0.000020,0.000004


In [5]:
merged = pd.read_csv(processed_dir / 'merged_energy_wealth.csv')
print('Merged dataset loaded:', merged.shape)

Merged dataset loaded: (160, 10)
